In [6]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [10]:
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

In [11]:
if not api_key:
    print('no api_key_found')
else: 
    print('API key found')

API key found


In [12]:
#creting instance of openai class
openai=OpenAI



In [ ]:
########################## EXTRACTING DATA FROM THE WEBSITE################################

headers={
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class website : 
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=headers)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title.string if soup.title else 'No title found'
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text=soup.body.get_text(separator="\n",strip=True)

In [20]:
ed=website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home 
 Connect Four 
 Outsmart 
 An arena that pits LLMs against each other in a battle of diplomacy and deviousness 
 About 
 Posts 
 Well, hi there. 
 I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production ( 
 very 
 amateur) and losing myself in 
 Hacker News 
 , nodding my head sagely to things I only half understand. 
 I’m the co-founder and CTO of 
 Nebula.io 
 . We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt, 
 acquired in 2021 
 . 
 We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve 
 patented 
 our matching model, and our award-winning platform has happy custom